In [ ]:
import cv2
import torch

import numpy as np
import pandas as pd

from PIL import Image
from deepface import DeepFace

from tqdm import tqdm
from IPython.display import clear_output

import os
import warnings

warnings.filterwarnings('ignore')
clear_output()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

path_to_original_dataset = "./FaceForensics++"
path_to_processed_dataset = "./ProcessedDataset"

folders = os.listdir(path_to_original_dataset)
folders.remove('csv')

FRAMES_STEP = 5

N_FRAMES = 30

IMAGE_WIDTH = 720
IMAGE_HEIGHT = 720

### Get frames from images

In [3]:
def get_video_frames(path, step=FRAMES_STEP, max_frames=N_FRAMES):
    video = cv2.VideoCapture(path)

    if not video.isOpened():
        assert print(f"Error occured while opening a video - {path}\n")

    frames = []
    count = 0
    while True:
        ret, frame = video.read()
        if not ret or len(frames) == max_frames:
            break

        if (count % step == 0):
            frames.append(frame)
        count += 1
    
    video.release()
    return np.array(frames)

In [5]:
for folder in folders:
    path_to_videos = os.path.join(path_to_original_dataset, folder)
    path_to_save = os.path.join(path_to_processed_dataset, folder)
    os.makedirs(path_to_save, exist_ok=True)

    videos = os.listdir(path_to_videos)

    for video in tqdm(videos, desc=f"Processing {folder}"):
        frames = get_video_frames(os.path.join(path_to_videos, video))

        # detect face and crop frame
        frames_to_save = []
        for frame in frames:
            # make PIL image from array and resize it
            pil_frame = Image.fromarray(frame)
            pil_frame = pil_frame.resize((IMAGE_WIDTH, IMAGE_HEIGHT))

            # detect face
            locations = DeepFace.extract_faces(np.array(pil_frame), detector_backend='yolov8', enforce_detection=False)
            if (len(locations) != 0):
                most_conf_ind = np.argmax([locations[i]['confidence'] for i in range(len(locations))])

                # in case that face not found on a frame
                if (locations[most_conf_ind]['confidence'] == 0):
                    continue
                
                face_as_np_array = np.array(locations[most_conf_ind]['face'] * 255, dtype=np.uint8)
                face = Image.fromarray(face_as_np_array)

                path = os.path.join(path_to_save, video[:-4], "frame_" + str(len(frames_to_save)) + ".jpg")
                frames_to_save.append((face, path))                
        
        # save if detected enough frames
        if (len(frames_to_save) == N_FRAMES):
            os.makedirs(os.path.join(path_to_save, video[:-4]))
            for frame, path in frames_to_save:
                frame.save(path)

Processing FaceSwap: 100%|██████████| 1000/1000 [1:01:12<00:00,  3.67s/it]
